In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import geopandas
import pandas
import rasterio

from affine import Affine
from rasterio import features
from snail.damages import PiecewiseLinearDamageCurve

In [ ]:
value_150ss_tif = "../results/input/giri/THA/bem_5x5_valfis_res__THA.tif"
volume_3ss_tif = "../results/input/ghsl/THA/GHS_BUILT_V_E2020_GLOBE_R2023A_4326_3ss_V1_0__THA.tif"
flood_1ss_tif = "../results/input/footprints/JBA/Raster/TH_FLRF_ChaoPhraya2011_RD_01.tif"

In [ ]:
def clip_array(arr, block_size):
    clip_rows = (arr.shape[0] - (arr.shape[0] % block_size))
    clip_cols = (arr.shape[1] - (arr.shape[1] % block_size))

    clipped = arr[0:clip_rows, 0:clip_cols]
    return clipped

def resample_sum(arr, block_size):
    nblocks_0 = arr.shape[0] // block_size
    nblocks_1 = arr.shape[1] // block_size

    blocks = arr.reshape(nblocks_0, block_size, nblocks_1, block_size)

    return np.sum(blocks, axis=(1, 3))

In [ ]:
# d = np.arange(12).reshape((3,4))
# d, resample_sum(d, 2)

In [ ]:
def repeat_2d(arr, block_size):
    """Repeat each element from a 2d array, so each value fills a (block_size x block_size) area
    """
    return np.repeat(np.repeat(arr, block_size, axis=0), block_size, axis=1)

# repeat_2d(d, 2)

In [ ]:
def read_ds(ds, band=1, replace_nodata=False, nodata_fill=0):
    data = ds.read(band)
    if replace_nodata:
        data = np.where(data == ds.nodata, nodata_fill, data)
    return data

In [ ]:
with rasterio.open(value_150ss_tif) as value_150ss_ds:
    value_150ss_all = read_ds(value_150ss_ds, replace_nodata=True)

with rasterio.open(volume_3ss_tif) as volume_3ss_ds:
    volume_3ss_all = read_ds(volume_3ss_ds, replace_nodata=True)

In [ ]:
def to_int(a):
    return np.floor(a).astype(int)

# lon, lat of volume_3ss top left
volume_3ss_all_ul_xy = volume_3ss_ds.transform * (0,0)
# col, row in value_150ss_all, inset one extra
value_150ss_ul_cr = to_int(~value_150ss_ds.transform * (volume_3ss_all_ul_xy)) + 1
# lon, lat of that value_150ss_all pixel - this is our new top left
ul_xy_150ss = value_150ss_ds.transform * value_150ss_ul_cr
# col, row in volume_3ss_all
volume_3ss_ul_cr = to_int(~volume_3ss_ds.transform * ul_xy_150ss)
# lon, lat of that volume_3ss_all pixel - new top left for 3ss purposes (tiny bit offset)
ul_xy_3ss = volume_3ss_ds.transform * volume_3ss_ul_cr
ul_xy_150ss, ul_xy_3ss

In [ ]:
# Clip out volume array
col_idx, row_idx = volume_3ss_ul_cr
volume_3ss = volume_3ss_all[row_idx:, col_idx:]
volume_3ss = clip_array(volume_3ss, 50)
# Resample volume to coarse-scale, "sum"
volume_150ss = resample_sum(volume_3ss, 50)
volume_150ss.shape

In [ ]:
# Adapt transform to new top-left and resolution
a,b,c,d,e,f = volume_3ss_ds.transform[:6]
t_150ss = Affine(
    a * 50,
    b,
    ul_xy_150ss[0],
    d,
    e * 50,
    ul_xy_150ss[1]
)
t_3ss = Affine(
    a,
    b,
    ul_xy_3ss[0],
    d,
    e,
    ul_xy_3ss[1]
)
t_150ss, t_3ss

In [ ]:
col_idx, row_idx = value_150ss_ul_cr
ncols, nrows = volume_150ss.shape
value_150ss = value_150ss_all[col_idx:col_idx+ncols, row_idx:row_idx+nrows]

In [ ]:
value_150ss.shape, volume_150ss.shape

In [ ]:
with rasterio.open("../results/input/giri/THA/vol_150ss.tif", 'w',
    driver='GTiff',
    height=volume_150ss.shape[0],
    width=volume_150ss.shape[1],
    count=1,
    dtype='float64',
    crs='+proj=latlong',
    transform=t_150ss
) as ds:
    ds.write(volume_150ss, indexes=1)

In [ ]:
with rasterio.open("../results/input/giri/THA/vol_3ss.tif", 'w',
    driver='GTiff',
    height=volume_3ss.shape[0],
    width=volume_3ss.shape[1],
    count=1,
    dtype=volume_3ss.dtype,
    crs='+proj=latlong',
    transform=t_3ss
) as ds:
    ds.write(volume_3ss, indexes=1)

In [ ]:
if value_150ss.shape != volume_150ss.shape:
    print("CHKS", value_150ss.shape, volume_150ss.shape)
    assert False

# Calculate value per unit volume
# value_per_volume_150ss = value_150ss / volume_150ss
value_per_volume_150ss = np.divide(value_150ss, volume_150ss, out=np.zeros_like(value_150ss), where=volume_150ss!=0)
# Resample to fine-scale value per volume, "nearest"
value_per_volume_3ss = repeat_2d(value_per_volume_150ss, 50)
# Calculate fine-scale value
value_3ss = value_per_volume_3ss * volume_3ss

In [ ]:
with rasterio.open("../results/input/giri/THA/val_vol_150ss.tif", 'w',
    driver='GTiff',
    height=value_per_volume_150ss.shape[0],
    width=value_per_volume_150ss.shape[1],
    count=1,
    dtype=value_per_volume_150ss.dtype,
    crs='+proj=latlong',
    transform=t_150ss
) as ds:
    # Write to window
    ds.write(value_per_volume_150ss, indexes=1)

In [ ]:
with rasterio.open("../results/input/giri/THA/val_vol_3ss.tif", 'w',
    driver='GTiff',
    height=value_per_volume_3ss.shape[0],
    width=value_per_volume_3ss.shape[1],
    count=1,
    dtype=value_per_volume_3ss.dtype,
    crs='+proj=latlong',
    transform=t_3ss
) as ds:
    # Write to window
    ds.write(value_per_volume_3ss, indexes=1)

In [ ]:
with rasterio.open("../results/input/giri/THA/val_3ss.tif", 'w',
    driver='GTiff',
    height=value_3ss.shape[0],
    width=value_3ss.shape[1],
    count=1,
    dtype=value_3ss.dtype,
    crs='+proj=latlong',
    transform=t_3ss
) as ds:
    # Write to window
    ds.write(value_3ss, indexes=1)

## Flood intersection

In [ ]:
with rasterio.open(flood_1ss_tif, 'r') as flood_1ss_ds:
    flood_1ss = read_ds(flood_1ss_ds, replace_nodata=True)

In [ ]:
flood_1ss_ds.transform

In [ ]:
# lon, lat of footprint top left
flood_1ss_ul_xy = flood_1ss_ds.transform * (0,0)
# col, row in value_3ss
t_3ss_ul_cr = to_int(~t_3ss * (flood_1ss_ul_xy))
# lon, lat of that pixel - this is our new top left
footprint_ul_xy_3ss = t_3ss * t_3ss_ul_cr
# col, row in flood_1ss
flood_1ss_ul_cr = to_int(~flood_1ss_ds.transform * footprint_ul_xy_3ss)
# lon, lat of that flood_1ss pixel - new top left for 1ss purposes (tiny bit offset)
ul_xy_1ss = flood_1ss_ds.transform * flood_1ss_ul_cr
flood_1ss_ul_xy, footprint_ul_xy_3ss, ul_xy_1ss

# TODO should new top left be greater, not less, in both x and y values?

In [ ]:
# clip to match coarser array extent
flood_1ss_clipped = clip_array(flood_1ss, 3)
flood_1ss_height, flood_1ss_width = flood_1ss_clipped.shape

In [ ]:
# lon, lat of footprint lower right
flood_1ss_lr_xy = flood_1ss_ds.transform * (flood_1ss_width, flood_1ss_height)
# col, row in value_3ss
t_3ss_lr_cr = to_int(~t_3ss * (flood_1ss_lr_xy))

In [ ]:
ulc, ulr = t_3ss_ul_cr
lrc, lrr = t_3ss_lr_cr
footprint_value_3ss = value_3ss[ulr:lrr, ulc:lrc]

footprint_value_1ss = repeat_2d(footprint_value_3ss, 3) / 9

In [ ]:
building_flood_depth_damage_curve = PiecewiseLinearDamageCurve.from_csv(
    "../bundled_data/damage_curves/flood/residential_asia.csv",
    intensity_col="inundation_depth_(m)",
    damage_col="damage_fraction")

In [ ]:
if footprint_value_1ss.shape != flood_1ss_clipped.shape:
    print("CHKS", footprint_value_1ss.shape, flood_1ss_clipped.shape)
    assert False

damage_fraction_1ss = building_flood_depth_damage_curve.damage_fraction(flood_1ss_clipped)

damage_value_1ss = footprint_value_1ss * damage_fraction_1ss

In [ ]:
# Adapt transform to new top-left and resolution
a,b,c,d,e,f = flood_1ss_ds.transform[:6]
t_1ss = Affine(
    a,
    b,
    ul_xy_1ss[0],
    d,
    e,
    ul_xy_1ss[1]
)
t_1ss

In [ ]:
with rasterio.open("../results/input/giri/THA/dmg_frac_1ss.tif", 'w',
    driver='GTiff',
    height=damage_fraction_1ss.shape[0],
    width=damage_fraction_1ss.shape[1],
    count=1,
    dtype=damage_fraction_1ss.dtype,
    crs='+proj=latlong',
    transform=t_1ss
) as ds:
    ds.write(damage_fraction_1ss, indexes=1)

In [ ]:
with rasterio.open("../results/input/giri/THA/dmg_val_1ss.tif", 'w',
    driver='GTiff',
    height=damage_value_1ss.shape[0],
    width=damage_value_1ss.shape[1],
    count=1,
    dtype=damage_value_1ss.dtype,
    crs='+proj=latlong',
    transform=t_1ss
) as ds:
    ds.write(damage_value_1ss, indexes=1)

In [ ]:
damage_value_1ss.sum() / 1e9

In [ ]:
value_150ss.sum() / 1e9, value_3ss.sum() / 1e9

In [ ]:
footprint_value_3ss.sum() / 1e9, footprint_value_1ss.sum() / 1e9

In [ ]:
with rasterio.open("../results/input/giri/THA/nres_dmg_val_1ss.tif") as nres_dmg_val_1ss_ds:
    nres_dmg_val_1ss = read_ds(nres_dmg_val_1ss_ds)
nres_dmg_val_1ss.sum() / 1e9

In [ ]:
"""
ADM1 damage values:

    exactextract \
        -p ../../admin-boundaries/tha_adm1.shp \
        -r dmg_val_1ss.tif \
        -f GID_1 \
        -s sum \
        -o dmg_val_1ss.csv

ADM1 total built volume:

    exactextract \
        -p ../../admin-boundaries/tha_adm1.shp \
        -r ../../ghsl/THA/GHS_BUILT_V_E2020_GLOBE_R2023A_4326_3ss_V1_0__THA.tif \
        -f GID_1 \
        -s sum \
        -o ghs_built_v_3ss.csv
"""

In [ ]:
adm1_vol = pandas.read_csv("input/giri/THA/ghs_built_v_3ss.csv") \
    .rename(columns={"sum": "built_volume"})

In [ ]:
adm1 = geopandas.read_file("input/admin-boundaries/tha_adm1.shp") \
    .merge(adm1_vol, on="GID_1")[["GID_1", "NAME_1", "built_volume", "geometry"]]

In [ ]:
adm1

In [ ]:
with rasterio.open("input/ghsl/THA/GHS_BUILT_V_E2020_GLOBE_R2023A_4326_3ss_V1_0__THA.tif") as vol_3ss_ds:
    vol_3ss = vol_3ss_ds.read(1)

In [ ]:
def rasterize(gdf, column, template_ds):
    return features.rasterize(
        ((f['geometry'], f['properties'][column]) for f in gdf.__geo_interface__['features']),
        out_shape=template_ds.shape,
        transform=template_ds.transform
    )

vol_adm1_3ss = rasterize(adm1, 'built_volume', vol_3ss_ds)

In [ ]:
plt.imshow(vol_adm1_3ss)

In [ ]:
adm1_gva = pandas.read_csv("/data/incoming/wenz-2023-dose-reported-subnational-output/DOSE_V2_THA.csv")
adm1_gva["ag_grp"] = adm1_gva["pop"] * adm1_gva.ag_grp_pc_usd
adm1_gva["man_grp"] = adm1_gva["pop"] * adm1_gva.man_grp_pc_usd
adm1_gva["serv_grp"] = adm1_gva["pop"] * adm1_gva.serv_grp_pc_usd

adm1_gva = geopandas.read_file("input/admin-boundaries/tha_adm1.shp") \
    .merge(adm1_gva, on="GID_1")[["GID_1", "NAME_1", "ag_grp", "man_grp", "serv_grp", "geometry"]]

In [ ]:
adm1_gva.drop(columns="geometry").to_csv("input/giri/THA/DOSE_V2_THA_rgva.csv")
adm1_gva.to_file("input/giri/THA/DOSE_V2_THA_rgva.gpkg", driver="GPKG")

In [ ]:
adm1_gva_ag_3ss = rasterize(adm1_gva, "ag_grp", vol_3ss_ds)
adm1_gva_man_3ss = rasterize(adm1_gva, "man_grp", vol_3ss_ds)
adm1_gva_serv_3ss = rasterize(adm1_gva, "serv_grp", vol_3ss_ds)

In [ ]:
def zero_divide(a, b):
    return np.divide(a, b, out=np.zeros_like(a, dtype='float64'), where=(b!=0))

In [ ]:
gva_ag_3ss = zero_divide(vol_3ss, vol_adm1_3ss) * adm1_gva_ag_3ss
gva_man_3ss = zero_divide(vol_3ss, vol_adm1_3ss) * adm1_gva_man_3ss
gva_serv_3ss = zero_divide(vol_3ss, vol_adm1_3ss) * adm1_gva_serv_3ss

In [ ]:
def write_ds(fname, data, transform):
    with rasterio.open(fname, 'w',
        driver='GTiff',
        height=data.shape[0],
        width=data.shape[1],
        count=1,
        dtype=data.dtype,
        crs='+proj=latlong',
        transform=transform
    ) as ds:
        ds.write(data, indexes=1)

In [ ]:
write_ds("input/giri/THA/gva_ag_3ss.tif", gva_ag_3ss, vol_3ss_ds.transform)
write_ds("input/giri/THA/gva_man_3ss.tif", gva_man_3ss, vol_3ss_ds.transform)
write_ds("input/giri/THA/gva_serv_3ss.tif", gva_serv_3ss, vol_3ss_ds.transform)

In [ ]:
gva_ag_1ss = repeat_2d(gva_ag_3ss, 3) / 9
gva_man_1ss = repeat_2d(gva_man_3ss, 3) / 9
gva_serv_1ss = repeat_2d(gva_serv_3ss, 3) / 9

In [ ]:
# TODO figure out transform, check we're on the right grid, write out to files
# TODO compare with damage fraction, write out interruption
# TODO calculate per day, sum back to zonal stats
# TODO check totals (re-aggregate after disaggregation) maybe rescale???

In [ ]:
a,b,c,d,e,f = vol_3ss_ds.transform[:6]
gva_t_1ss = Affine(
    a / 3,
    b,
    c,
    d,
    e / 3,
    f
)
gva_t_1ss

In [ ]:
write_ds("input/giri/THA/gva_ag_1ss.tif", gva_ag_1ss, gva_t_1ss)
write_ds("input/giri/THA/gva_man_1ss.tif", gva_man_1ss, gva_t_1ss)
write_ds("input/giri/THA/gva_serv_1ss.tif", gva_serv_1ss, gva_t_1ss)

In [ ]:
"""
gdalwarp -te 99.2393056 13.2781945 101.5259723 17.6765279 gva_man_1ss.tif gva_man_1ss_clipped.tif
gdal_calc.py -A nres_dmg_frac_1ss.tif -B gva_man_1ss_clipped.tif --outfile=disruption_man_1ss.tif --calc="(A>0.1)*B"


gdalwarp -te 99.2393056 13.2781945 101.5259723 17.6765279 gva_ag_1ss.tif gva_ag_1ss_clipped.tif
gdal_calc.py -A nres_dmg_frac_1ss.tif -B gva_ag_1ss_clipped.tif --outfile=disruption_ag_1ss.tif --calc="(A>0.1)*B"


gdalwarp -te 99.2393056 13.2781945 101.5259723 17.6765279 gva_serv_1ss.tif gva_serv_1ss_clipped.tif
gdal_calc.py -A nres_dmg_frac_1ss.tif -B gva_serv_1ss_clipped.tif --outfile=disruption_serv_1ss.tif --calc="(A>0.1)*B"


for sector in serv ag man
    exactextract \
        -p ../../admin-boundaries/tha_adm1.shp \
        -r disruption_{$sector}_1ss.tif \
        -f GID_1 \
        -s sum \
        -o disruption_{$sector}_1ss.csv
end
"""

In [ ]:
"""
gdalwarp -te 99.2393056 13.2781945 101.5259723 17.6765279  ../../footprints/JBA/Raster/TH_FLRF_ChaoPhraya2011_RD_01.tif ../../footprints/JBA/Raster/TH_FLRF_ChaoPhraya2011_RD_01_clipped.tif
for sector in serv ag man
    gdal_calc.py \
        -A ../../footprints/JBA/Raster/TH_FLRF_ChaoPhraya2011_RD_01_clipped.tif \
        -B gva_{$sector}_1ss_clipped.tif \
        --outfile=disruption_0.3m_{$sector}_1ss.tif \
        --calc="(A>0.3)*B"
    exactextract \
        -p ../../admin-boundaries/tha_adm1.shp \
        -r disruption_0.3m_{$sector}_1ss.tif \
        -f GID_1 \
        -s sum \
        -o disruption_0.3m_{$sector}_1ss.csv
end

for sector in serv ag man
    gdal_calc.py \
        -A nres_dmg_frac_1ss.tif \
        -B gva_{$sector}_1ss_clipped.tif \
        --outfile=disruption_dmg_{$sector}_1ss.tif \
        --calc="A*B"
    exactextract \
        -p ../../admin-boundaries/tha_adm1.shp \
        -r disruption_dmg_{$sector}_1ss.tif \
        -f GID_1 \
        -s sum \
        -o disruption_dmg_{$sector}_1ss.csv
end
"""